# CS280 Programming Assignment 4
__Cocktail Party Problem__<br>
<br>
Compiler: Python 3.6.5<br>
OS: Windows 7 64-bit

Answers to questions:<br>
a) What is the sampling rate for the input files?<br>
>The sampling rate used is 22050 Hz. This value was returned by the method used for reading wav files, scipy.io.wavfile.read.<br>

b) Is centering necessary?<br>
>Yes, centering is required. As can be seen on section __*3b*__, FastICA returns an error when _whiten_ is disabled and uncentered data is inputted to FastICA.
Therefore we conclude that centering is necessary for ICA.

c) Is whitening required?<br>
>Yes, whitening is required. As can be seen on section __*3c*__, FastICA does not converge when _whiten_ is disabled, even if the input data is centered.
Therefore we conclude that whitening is necessary for ICA.

d) What is the appropriate contrastive function G(y)?<br>
>To choose which is the most appropriate G(y) to use, we compute the residuals for each G(y) option and choose the one that yields the least residuals.<br>Shown in sections __*3d*__, __*3e*__, and __*3f*__, the __exp__ function yielded the least residuals, and therefore is chosen as the most appropriate.

e) What are the residuals of the reconstructed mixture signals?<br>
>As shown in section 5: (May vary for every run)
Residuals for G(y)=exp:  [0.30384944 0.94084872 0.63921146 0.10750831 0.19435049]

## 1. Load the audio files mic1.wav to mic5.wav found in the folder Audio_Data. 
These files are synchronized audio recordings captured by five microphones positioned at five different locations

In [1]:
import scipy.io.wavfile as wav
import os

data_path = './Audio_Data'

sampling_freq, mic1_data = wav.read(os.path.join(data_path, 'mic1.wav'))
sampling_freq, mic2_data = wav.read(os.path.join(data_path, 'mic2.wav'))
sampling_freq, mic3_data = wav.read(os.path.join(data_path, 'mic3.wav'))
sampling_freq, mic4_data = wav.read(os.path.join(data_path, 'mic4.wav'))
sampling_freq, mic5_data = wav.read(os.path.join(data_path, 'mic5.wav'))

In [2]:
print("The sampling rate used for the input files is :", sampling_freq, "Hz.")

The sampling rate used for the input files is : 22050 Hz.


## 2. Form the mixture matrix X from the input files

In [3]:
import numpy as np

X = np.array([mic1_data, mic2_data, mic3_data, mic4_data, mic5_data])

# standardize the data
X = np.transpose(X)/X.std(axis=1)


In [4]:
X.shape

(191258, 5)

Optional: play sound

In [5]:
# import sounddevice as sd
# sd.play(0.1*X[:, 0], sampling_freq)

## 3. Invoke the appropriate ICA command
that "unmixes" the five independent components from the mixture of audio signals. Experiment on the ff:
* appropriate input sampling rate
* whether centering is required
* whether whitening is necessary
* appropriate contrastive function G(y)

First, let's define a function that plays the unmixed components which are outputted by FastICA:

In [6]:
import sounddevice as sd
import time

def play_independent_components(S, sampling_freq):
    print('========================================================================================')
    print('INFO: Turn your volume up to listen to the separated components! (Earphones recommended)')
    print('========================================================================================')
    S = np.transpose(S)
    for index, component in enumerate(S):
        print('Playing component %d:' % (index+1))
        sd.play(component, sampling_freq)
        time.sleep(len(component)/sampling_freq + 1)
    print('Done!')
    


Before trying anything out, let's check if our data is already centered:

In [7]:
means = np.mean(X, axis=0)
print('means: ', means)

means:  [-4.78217954e-08 -1.79852265e-07 -7.77710164e-08  4.03623969e-09
 -3.21318801e-08]


The means of the mixed sound files seem to be very close to zero. Therefore we can consider the data centered by default.

#### 3.a Try default settings
* centering: yes
* whitening: yes
* G(y): logcosh (default)

In [8]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=X.shape[-1])
S = ica.fit_transform(X)

In [9]:
play_independent_components(50*S, sampling_freq)

INFO: Turn your volume up to listen to the separated components! (Earphones recommended)
Playing component 1:
Playing component 2:
Playing component 3:
Playing component 4:
Playing component 5:
Done!


#### 3.b Try removing centering
* centering: no
* whitening: yes
* G(y): logcosh (default)

Add random offsets to the mixture vectors so that they are not zero-centered

In [10]:
X_not_centered = X - 10*np.random.uniform(low=-5, high=5, size=(1, X.shape[-1]))
print('means: ', np.mean(X_not_centered, axis=0))


means:  [-25.93151835  36.71859269   6.9292148  -10.71708155   3.83075995]


If the "whiten" option in FastICA is set to True, it will perform both centering and whitening on the mixture signals.<br>
Since we want to remove only the centering, we would have to manually implement whitening.<br>
<br>
We do that below by implementing our own method.<br>

From the Program Assignment handouts, the whitening matrix V is computed as:<br>
$$ V = 2(C_x)^{-\frac{1}{2}} $$
where $C_x$ is the covariance matrix of X.

In [11]:
def whiten(X):
    X = np.transpose(X)
    Cx = np.cov(X)
    
    V = 2*(1/np.sqrt(np.absolute(Cx)))
    
    X_whitened = np.matmul(V, X)
    return np.transpose(X_whitened)

In [12]:
X_not_centered = whiten(X_not_centered)
print(X_not_centered.shape)
ica = FastICA(n_components = X_not_centered.shape[-1], whiten=False)

(191258, 5)


If we run the line below, FastICA will return an error.
We take this to mean that the FastICA is unable to take un-centered data.<br>Therefore centering is required for ICA.

In [13]:
# S = ica.fit_transform(X_not_centered)

####  3.c Try removing whitening
* centering: yes
* whitening: no
* G(y): logcosh (default)

First let's check if the data is already centered:

In [14]:
print('means: ', np.mean(X, axis=0))

means:  [-4.78217954e-08 -1.79852265e-07 -7.77710164e-08  4.03623969e-09
 -3.21318801e-08]


The data seems centered at zero already. Now we perform FastICA with _whiten_ set to False:

In [15]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=X.shape[-1], whiten=False)
S = ica.fit_transform(X)

C:\anaconda3\lib\site-packages\sklearn\decomposition\fastica_.py:295: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn('Ignoring n_components with whiten=False.')
C:\anaconda3\lib\site-packages\sklearn\decomposition\fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In the results above, FastICA did not converge when whiten was set to False.
From this, we conclude that whitening is necessary for ICA.

In the next few parts, we try to vary the G(y) function used. For comparison purposes, we define the method below that computes the residuals.<br>
<br>


In [16]:
from sklearn.metrics import mean_squared_error

def compute_residuals(X, X_recon):
    errors = np.zeros((X.shape[-1]))
    for i in range(X.shape[-1]):
        errors[i] = mean_squared_error(X[:, i], X_recon[:, i])
    return errors

#### 3.d Try varying G(y): logcosh
* centering: yes
* whitening: yes
* G(y): logcosh (default)

In [17]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=X.shape[-1])
S = ica.fit_transform(X)

In [18]:
A = ica.mixing_
X_recon = np.matmul(S, A)

residuals_logcosh = compute_residuals(X, X_recon)
print('Residuals for G(y)=logcosh: ', residuals_logcosh)
print('mean = ', np.mean(residuals_logcosh))

Residuals for G(y)=logcosh:  [0.9095602  1.56944382 2.33119916 0.97037339 2.07436858]
mean =  1.5709890294612303


#### 3.e Try varying G(y): exp
* centering: yes
* whitening: yes
* G(y): exp

In [19]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=X.shape[-1], whiten=True, fun='exp')
S = ica.fit_transform(X)

In [20]:
A = ica.mixing_
X_recon = np.matmul(S, A)

residuals_exp = compute_residuals(X, X_recon)
print('Residuals for G(y)=exp: ', residuals_exp)
print('mean = ', np.mean(residuals_exp))

Residuals for G(y)=exp:  [0.41853667 1.48338466 0.80316792 2.12995522 1.63364625]
mean =  1.2937381455162444


#### 3.f Try varying G(y): cube
* centering: yes
* whitening: yes
* G(y): cube

In [21]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=X.shape[-1], whiten=True, fun='cube')
S = ica.fit_transform(X)

In [22]:
A = ica.mixing_
X_recon = np.matmul(S, A)

residuals_cube = compute_residuals(X, X_recon)
print('Residuals for G(y)=cube: ', residuals_cube)
print('mean = ', np.mean(residuals_cube))

Residuals for G(y)=cube:  [0.33465708 0.62956124 1.22618125 2.20526345 1.45503601]
mean =  1.1701398065934392


## 4. Save the independent components
as audio files in wav format. Label them as shat[1-5].wav

Best results when G(y) is exp:

In [23]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=X.shape[-1], whiten=True, fun='exp')
S = ica.fit_transform(X)

In [24]:
S.shape

(191258, 5)

In [25]:
import scipy.io.wavfile as wav

for i, unmixed in enumerate(np.transpose(S)):
    filename = 'shat%d.wav' % (i+1)
    wav.write(filename, sampling_freq, S[: , i])

## 5. Reconstruct the mixture signals
and measure the residuals for each one. Print out the residual values.

In [26]:
A = ica.mixing_
X_recon = np.matmul(S, A)

residuals_exp = compute_residuals(X, X_recon)
print('Residuals for G(y)=exp: ', residuals_exp)

Residuals for G(y)=exp:  [2.55297451 0.35472969 3.91322351 1.03906896 0.71446708]


## 6. Save the reconstructed mixture signals
as audio files in wav format. Label them as recon[1-5].wav

In [27]:
X_recon.shape

(191258, 5)

In [28]:
import scipy.io.wavfile as wav

for i, unmixed in enumerate(np.transpose(X_recon)):
    filename = 'recon%d.wav' % (i+1)
    wav.write(filename, sampling_freq, X_recon[: , i])